In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
import math
from IPython.display import FileLink


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    #for filename in filenames: print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames: 
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.


/kaggle/input
/kaggle/input/ecgdata
/kaggle/input/ecgdata/hf_round1_testA
/kaggle/input/ecgdata/hf_round1_train
/kaggle/input/prepare-round1-data
/kaggle/working/__notebook__.ipynb


In [2]:
!cp ../input/prepare-round1-data/disease_dict_round1.pickle .

from random import shuffle
import gzip
round1_add = '../input/prepare-round1-data/tianchi_round1.gz'
round1_f = gzip.open(round1_add,'rt')
datas = []
for line in round1_f:
    datas.append(line)

shuffle(datas)

f = gzip.open('tianchi_round1.gz','wt+')
#f = open('tianchi_round1','w+')
for line in tqdm(datas):
    f.write(line)
    if line[-1] != '\n':
        f.write('\n')

#sum([1 for line in open('tianchi_round1')])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [3]:
"""def load_labels(path):
    #load labels
    labels = []
    with open(path,'r') as f:
        for count, line in tqdm(enumerate(f)):
            datas = line.split()
            diseases = []
            exist_age = 0
            exist_sex = 0
            if datas[1].isdigit():  exist_age = 1
            if datas[1] == 'MALE' or datas[1] == 'FEMALE' : exist_sex = 1
            if len(datas) > 2 and (datas[2] == 'MALE' or datas[2] == 'FEMALE') : exist_sex = 1
            for disease in datas[1+exist_age+exist_sex:]:
                diseases.append(disease_dict[disease])
            labels.append(diseases)

    #transfer labels to one-hot
    hot_labels = []
    for label in labels:
        hot = [0]*len(disease_dict)
        for disease in label:
            hot[disease] = 1
        hot_labels.append(hot)
    hot_labels = torch.FloatTensor(hot_labels)
    hot_labels.shape

    num_dict = {}
    for dis in labels:
        for di in dis:
            num_dict[di] = num_dict.get(di,0) + 1
    #dict(sorted(num_dict.items()))
    return labels, hot_labels, num_dict
labels, hot_labels, num_dict = load_labels('/kaggle/input/ecgdata/hf_round1_label.txt')
#labels, hot_labels, num_dict = load_labels('/kaggle/input/hf-round2-train/hf_round2_train.txt')

#load diseases dict
import pickle
def load_disease_dict(path):
    disease_dict = {}
    with open(path,'r') as dict_f:
        for count, line in enumerate(dict_f):
            disease_dict[line.replace('\n','')] = count
    return disease_dict
disease_dict = load_disease_dict('/kaggle/input/ecgdata/hf_round1_arrythmia.txt')
pickle.dump(disease_dict,open('disease_dict_round1.pickle','wb+'))
print(disease_dict)
‘’‘
print({k: v for k, v in sorted(num_dict.items(), key=lambda item: item[1], reverse=True)})
for key in disease_dict:
    index = disease_dict[key]
    disease_dict[key] = num_dict[index]
print({k: v for k, v in sorted(disease_dict.items(), key=lambda item: item[1], reverse=True)})
’‘’

to_123 = {}
to_123['窦性心律'] = 82
to_123['T波改变'] = -1
to_123['窦性心动过缓'] = 81
to_123['ST段改变'] = 78
to_123['正常ECG'] = -1
to_123['左心室高电压'] = -1
to_123['ST-T改变'] = 61
to_123['窦性心动过速'] = 83
to_123['临界ECG'] = -1
to_123['QRS低电压'] = 51
to_123['房性早搏'] = -1
to_123['电轴左偏'] = 46
to_123['电轴右偏'] = 75
to_123['心房颤动'] = 8
to_123['异常ECG'] = -1
to_123['室性早搏'] = 68
to_123['完全性右束支传导阻滞'] = 24
to_123['窦性心律不齐'] = 80
to_123['左心室肥大'] = 49
to_123['右束支传导阻滞'] = 76
to_123['一度房室传导阻滞'] = -1
to_123['快心室率'] = -1
to_123['不完全性右束支传导阻滞'] = 33
to_123['非特异性T波异常'] = -1
to_123['QT间期延长'] = 69
to_123['左前分支传导阻滞'] = 42
to_123['非特异性ST段异常'] = -1
to_123['差异性传导'] = -1
to_123['非特异性ST段与T波异常'] = 61
to_123['起搏心律'] = 63
to_123['短PR间期'] = 79
to_123['下壁异常Q波'] = 71
to_123['快室率心房颤动'] = 8
to_123['逆钟向转位'] = -1
to_123['室内差异性传导'] = -1
to_123['早期复极化'] = 27
to_123['二联律'] = -1
to_123['室上性早搏'] = 87
to_123['顺钟向转位'] = -1
to_123['复极化异常'] = -1
to_123['未下传的房性早搏'] = -1
to_123['肺心病型'] = -1
to_123['慢心室率'] = -1
to_123['短串房性心动过速'] = 13
to_123['非特异性室内传导延迟'] = -1
to_123['右心房扩大'] = 74
to_123['左束支传导阻滞'] = 47
to_123['前间壁R波递增不良'] = 72
to_123['右心室肥大'] = 77
to_123['房室传导延缓'] = 65
to_123['双分支传导阻滞'] = 19
to_123['非特异性室内传导阻滞'] = 60
to_123['肺型P波'] = -1
to_123['完全性左束支传导阻滞'] = -1
to_123['融合波'] = -1

f = open('ori_round1_label_to_123_lalel','w+')
for key in disease_dict:
    assert key in to_123
    out_str = [str(key), str(disease_dict[key]), str(to_123[key])]
    f.write('\t'.join(out_str) + '\n')
f.close()

ori_round1_label_to_123_lalel = []
for line in open('ori_round1_label_to_123_lalel'):
    ori_round1_label_to_123_lalel.append(line.strip())
'\n'.join(ori_round1_label_to_123_lalel)

def get_train_data(path):
    train_data = []
    for dirname, dirs, filenames in os.walk(path):
        #filenames = sorted(filenames)
        filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
        for file_count, filename in tqdm(enumerate(filenames)):
            address = os.path.join(dirname, filename)
            #if file_count == 10000: break
            with open(address,'r') as f:
                individual = []
                for line, v in enumerate(f):
                    if line == 0: continue
                    v = v.split()
                    #print(v)
                    individual.append([float(value) for value in v])
                    del v
                individual = torch.FloatTensor(individual).transpose(0,1)
                #train_data[file_count] = individual
                train_data.append(individual)
                del individual
                f.close()
            #break
    train_data = torch.stack(train_data)
    return train_data
#get_train_data('/kaggle/input/hf-round2-train/hf_round2_train')
#train_data = get_train_data('/kaggle/input/hf-round2-train/hf_round2_train')
train_data = get_train_data('/kaggle/input/ecgdata/hf_round1_train')

#add up to 12 leads
do_12leads = 1
if do_12leads:
    I = train_data[:,0]
    II = train_data[:,1]
    III=II-I
    aVR=-(I+II)/2
    aVL=I-II/2
    aVF=II-I/2

    III = III.unsqueeze(1)
    aVR = aVR.unsqueeze(1)
    aVL = aVL.unsqueeze(1)
    aVF = aVF.unsqueeze(1)
    train_data = torch.cat([train_data, III, aVR, aVL, aVF],dim=1)
    del I, II, III, aVR, aVL, aVF
train_data.shape

import base64
def write_data_to_file(output_path, datas, labels):
    key = 0
    with open(output_path,'w+') as f:
        for data,label in zip(datas,labels):
            output_str = [str(key), base64.b64encode(data.numpy().astype(np.float16)).decode("utf-8"),
                              base64.b64encode(label.numpy().astype(np.float16)).decode("utf-8"), '-1']
            f.write('\t'.join(output_str))
            #if key != 20035:
            if key != 24105:
                f.write('\n')
            key += 1
            #print(key)
write_data_to_file('tianchi_round1', train_data, hot_labels)

!tar -zcf tianchi_round1.gz tianchi_round1
!rm tianchi_round1
#exit(0)"""

'def load_labels(path):\n    #load labels\n    labels = []\n    with open(path,\'r\') as f:\n        for count, line in tqdm(enumerate(f)):\n            datas = line.split()\n            diseases = []\n            exist_age = 0\n            exist_sex = 0\n            if datas[1].isdigit():  exist_age = 1\n            if datas[1] == \'MALE\' or datas[1] == \'FEMALE\' : exist_sex = 1\n            if len(datas) > 2 and (datas[2] == \'MALE\' or datas[2] == \'FEMALE\') : exist_sex = 1\n            for disease in datas[1+exist_age+exist_sex:]:\n                diseases.append(disease_dict[disease])\n            labels.append(diseases)\n\n    #transfer labels to one-hot\n    hot_labels = []\n    for label in labels:\n        hot = [0]*len(disease_dict)\n        for disease in label:\n            hot[disease] = 1\n        hot_labels.append(hot)\n    hot_labels = torch.FloatTensor(hot_labels)\n    hot_labels.shape\n\n    num_dict = {}\n    for dis in labels:\n        for di in dis:\n           